In [5]:
import torch
import torchvision
import torchvision.models as models
from PIL import Image, ImageFile
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import gc; gc.enable()
import pandas as pd
from torch.utils.data import Dataset
import tensorflow as tf
from tensorflow import keras
from torchvision import transforms

In [14]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
# other imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

In [7]:
train_dir = '/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/xgboost_train_features/'

In [8]:
gc.collect()

142

In [9]:
from sklearn.model_selection import StratifiedKFold, KFold
import xgboost as xgb
params = {
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 5,
}

In [11]:
import xgbfir


# sub_preds = np.zeros(test_.shape[0])

feature_importance_df = pd.DataFrame()

In [13]:
for cycle in range(4):
    pth = train_dir+'train_features_'+str(cycle)+'.csv'
    lbls_pth = train_dir+'y_train_'+str(cycle)+'.csv'
    train = pd.read_csv(pth)
    y_train = pd.read_csv(lbls_pth)
    trn_x, trn_y, val_x, val_y = train_test_split(train, y_train, test_size = 0.2)
    dtrain = xgb.DMatrix(trn_x, trn_y)
    val_train = xgb.DMatrix(val_x, val_y)
    oof_preds = np.zeros(len(val_y))
    gc.collect()
    
    clf = xgb.train(params=params, dtrain=dtrain, num_boost_round=1500, early_stopping_rounds=100) 
    gc.collect()
    
    oof_preds = clf.predict(xgb.DMatrix(val_x))
#     sub_preds += clf.predict(xgb.DMatrix(test[train_cols])) / folds.n_splits
    
    xgbfir.saveXgbFI(clf, feature_names=trn_x.columns, OutputXlsxFile='ieee_xgbfir_%sFold.xlsx'%str(n_fold+1), MaxInteractionDepth=9, MaxHistograms=15)
    gc.collect()
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].index
    fold_importance_df["fscore"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].values
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    gc.collect()
    print(cohen_kappa_score(y_test, y_pred))
    print('\nFold %2d cohens kappa %.6f & std %.6f' %(cycle + 1, cohen_kappa_score(val_y, oof_preds[val_idx]), np.std([oof_preds])))
    gc.collect()

# print('\nCV cohens kappa score score %.6f & std %.6f' % (cohen_kappa_score(y_train, oof_preds), np.std((oof_preds))))


NameError: name 'train_test_split' is not defined

In [1]:
X = feat_train
bleh, y = np.where(labels_train)

xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(feat_test)

NameError: name 'feat_train' is not defined

In [ ]:

bleh, y_test = np.where(labels_test)
print(cohen_kappa_score(y_test, y_pred))